In [ ]:
# https://github.com/salesforce/LAVIS/blob/main/lavis/models/blip2_models/blip2_qformer.py#L260
decoder_input_ids = text_tokens.input_ids.clone()
decoder_input_ids[:, 0] = self.tokenizer.bos_token_id
labels = decoder_input_ids.masked_fill(
    decoder_input_ids == self.tokenizer.pad_token_id, -100
)

query_atts = torch.ones(query_tokens.size()[:-1], dtype=torch.long).to(
    image.device
)
attention_mask = torch.cat([query_atts, text_tokens.attention_mask], dim=1)
lm_output = self.Qformer(
    decoder_input_ids,
    attention_mask=attention_mask,
    past_key_values=query_output.past_key_values,
    return_dict=True,
    labels=labels,
)

loss_lm = lm_output.loss

注意forward的细微变化，这里是self.Qformer()，而不是self.Qformer.bert()。self.Qformer是BertLMHeadModel类。

这里在进行语言建模，在解码，只需要query_output的kv，不需要它的隐状态，所以past_key_values=query_output.past_key_values。

In [ ]:
# https://github.com/salesforce/LAVIS/blob/main/lavis/models/blip2_models/Qformer.py#L987
# BertLMHeadModel的forward方法
def forward(
    self,
    input_ids=None,
    attention_mask=None,
    position_ids=None,
    head_mask=None,
    query_embeds=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    labels=None,
    past_key_values=None,
    use_cache=True,
    output_attentions=None,
    output_hidden_states=None,
    return_dict=None,
    return_logits=False,
    is_decoder=True,
    reduction="mean",
):

In [ ]:
input_ids=decoder_input_ids
attention_mask=attention_mask
past_key_values=query_output.past_key_values
labels=labels

In [ ]:
# BertLMHeadModel的forward方法
# ...
if past_key_values is not None:
    query_embeds = None

outputs = self.bert(
    input_ids,
    attention_mask=attention_mask,
    position_ids=position_ids,
    head_mask=head_mask,
    query_embeds=query_embeds,
    encoder_hidden_states=encoder_hidden_states,
    encoder_attention_mask=encoder_attention_mask,
    past_key_values=past_key_values,
    use_cache=use_cache,
    output_attentions=output_attentions,
    output_hidden_states=output_hidden_states,
    return_dict=return_dict,
    is_decoder=is_decoder,
)

如果提供了past_key_values，说明是解码阶段，不需要query_embeds。

In [ ]:
# https://github.com/salesforce/LAVIS/blob/main/lavis/models/blip2_models/Qformer.py#L804
# BertModel类的forward方法
def forward(
    self,
    input_ids=None,
    attention_mask=None,
    position_ids=None,
    head_mask=None,
    query_embeds=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_values=None,
    use_cache=None,
    output_attentions=None,
    output_hidden_states=None,
    return_dict=None,
    is_decoder=False,
):

In [ ]:
input_ids=decoder_input_ids
attention_mask=attention_mask
past_key_values=query_output.past_key_values
is_decoder=True

In [ ]:
# BertModel类的forward方法
# ...
past_key_values_length = (
    past_key_values[0][0].shape[2] - self.config.query_length
    if past_key_values is not None
    else 0
)

past_key_values[0][0] 取出了第一个注意力头的第一个键。它的形状是 (batch_size, num_heads, seq_length, head_dim)。
past_key_values[0][0].shape[2] 取得了序列长度（seq_length）。

这段代码的目的是确定在当前推理步骤中，已经处理过的序列长度。这个长度减去当前查询序列长度，就得到了 past_key_values_length。

In [ ]:
# BertModel类的forward方法
query_length = query_embeds.shape[1] if query_embeds is not None else 0 

embedding_output = self.embeddings(
    input_ids=input_ids,
    position_ids=position_ids,
    query_embeds=query_embeds,
    past_key_values_length=past_key_values_length,
)

input_shape = embedding_output.size()[:-1]
batch_size, seq_length = input_shape
device = embedding_output.device


此query_length非彼self.config.query_length， query_length=0.

past_key_values_length用于推导position_ids的位置范围。

embedding_output是文本的embeddings。seq_length是文本的token数量。

In [ ]:
# BertModel类的forward方法
# We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
# ourselves in which case we just need to make it broadcastable to all heads.
if is_decoder:
    extended_attention_mask = self.get_extended_attention_mask(
        attention_mask,
        input_ids.shape,
        device,
        is_decoder,
        has_query=(query_embeds is not None),
    )
else:
    # ...

# get_extended_attention_mask方法
def get_extended_attention_mask(
    self,
    attention_mask=attention_mask,
    input_shape=input_ids.shape,
    device=device,
    is_decoder: True,
    has_query: False,
) -> Tensor:
    # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
    # ourselves in which case we just need to make it broadcastable to all heads.
    if attention_mask.dim() == 3:
        # ...
    else:
        if is_decoder:
            batch_size, seq_length = input_shape

            seq_ids = torch.arange(seq_length, device=device)
            causal_mask = (
                seq_ids[None, None, :].repeat(batch_size, seq_length, 1)
                <= seq_ids[None, :, None]
            )

            # add a prefix ones mask to the causal mask
            # causal and attention masks must have same type with pytorch version < 1.3
            causal_mask = causal_mask.to(attention_mask.dtype) 

In [ ]:
这是计算attention mask的地方，划重点。

causal_mask就是用于语言建模的mask，即某个位置只能注意到前面已经解码过的词。
- seq_ids[None, None, :] 通过增加两个维度变成形状为 (1, 1, seq_length) 的张量。
- eq_ids[None, None, :].repeat(batch_size, seq_length, 1) 通过重复操作变成形状为 (batch_size, seq_length, seq_length) 的张量。
- seq_ids[None, :, None] 通过增加两个维度变成形状为 (1, seq_length, 1) 的张量。
- seq_ids[None, None, :].repeat(batch_size, seq_length, 1) <= seq_ids[None, :, None] 通过比较操作生成一个布尔型张量，形状为 (batch_size, seq_length, seq_length)。

causal mask举例：
[[ True, False, False],
[ True,  True, False],
[ True,  True,  True]]
生成的 causal_mask 是一个三维布尔张量，形状为 (batch_size, seq_length, seq_length)。
对于每个位置 i，只有位置 i 及其之前的位置 j (即 j <= i) 才会是 True，其他位置是 False。这确保了在注意力计算中，位置 i 只能关注到位置 i 及其之前的位置，而不能看到未来的位置。

In [ ]:
# get_extended_attention_mask方法
if causal_mask.shape[1] < attention_mask.shape[1]:
    prefix_seq_len = attention_mask.shape[1] - causal_mask.shape[1]
    if has_query:  # UniLM style attention mask
        # ...
    causal_mask = torch.cat(
        [
            torch.ones(
                (batch_size, causal_mask.shape[1], prefix_seq_len),
                device=device,
                dtype=causal_mask.dtype,
            ),
            causal_mask,
        ],
        axis=-1,
    )
extended_attention_mask = (
    causal_mask[:, None, :, :] * attention_mask[:, None, None, :]
)

这部分是给图像queries的mask设置为1。prefix_seq_len就是图像queries的数量。

最后，将因果掩码和注意力掩码结合起来，生成一个扩展的注意力掩码 extended_attention_mask：
causal_mask[:, None, :, :] 为 causal_mask 的第二维度添加一个新维度，形状变为 (batch_size, 1, seq_length, seq_length)。
attention_mask[:, None, None, :] 为 attention_mask 的第二和第三维度添加新维度，形状变为 (batch_size, 1, 1, attention_mask_length)。
两者相乘，得到最终的 extended_attention_mask，形状为 (batch_size, 1, seq_length, attention_mask_length)。
这个扩展的注意力掩码将同时考虑因果性和实际的注意力掩码，从而确保在自注意力机制中正确地应用注意力权重。

In [ ]:
# BertModel类的forward方法
if encoder_hidden_states is not None:
    # ...
else:
    encoder_extended_attention_mask = None

# ...

encoder_outputs = self.encoder(
    embedding_output,
    attention_mask=extended_attention_mask,
    head_mask=head_mask,
    encoder_hidden_states=encoder_hidden_states,
    encoder_attention_mask=encoder_extended_attention_mask,
    past_key_values=past_key_values,
    use_cache=use_cache,
    output_attentions=output_attentions,
    output_hidden_states=output_hidden_states,
    return_dict=return_dict,
    query_length=query_length,
)

# BertEncoder的forward方法
def forward(
    self,
    hidden_states,
    attention_mask=None,
    head_mask=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_values=None,
    use_cache=None,
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True,
    query_length=0,
):

In [ ]:
hidden_states=embedding_output
attention_mask=extended_attention_mask
past_key_values=query_output.past_key_values
query_length=0

In [ ]:
# BertEncoder的forward方法
for i in range(self.config.num_hidden_layers):
    layer_module = self.layer[i]
        
    # ...

    layer_head_mask = head_mask[i] if head_mask is not None else None
    past_key_value = past_key_values[i] if past_key_values is not None else None 

    if getattr(self.config, "gradient_checkpointing", False) and self.training:
        # 略
    else:
        layer_outputs = layer_module(
            hidden_states,
            attention_mask,
            layer_head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            past_key_value,
            output_attentions,
            query_length,
        )
    # ...

# BertLayer的forward方法
def forward(
    self,
    hidden_states,
    attention_mask=None,
    head_mask=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_value=None,
    output_attentions=False,
    query_length=0,
):

In [ ]:
hidden_states=embedding_output
attention_mask=extended_attention_mask
past_key_values=query_output.past_key_values
query_length=0

In [ ]:
# BertLayer的forward方法
# decoder uni-directional self-attention cached key/values tuple is at positions 1,2
self_attn_past_key_value = (
    past_key_value[:2] if past_key_value is not None else None
)

self_attention_outputs = self.attention(
    hidden_states,
    attention_mask,
    head_mask,
    output_attentions=output_attentions,
    past_key_value=self_attn_past_key_value,
)

attention_output = self_attention_outputs[0]
outputs = self_attention_outputs[1:-1]

这里是实际计算attention的地方。q是hidden_states，kv是hidden_states+past_key_value。

In [ ]:
# BertLMHeadModel的forward方法
outputs = self.bert(...)

outputs是文本的self attention结果。

In [ ]:
# BertLMHeadModel的forward方法
prediction_scores = self.cls(sequence_output)

将每个位置的隐状态，映射到词表大小的logits.

In [ ]:
# BertLMHeadModel的forward方法
lm_loss = None
if labels is not None:
    # we are doing next-token prediction; shift prediction scores and input ids by one
    shifted_prediction_scores = prediction_scores[:, :-1, :].contiguous()
    labels = labels[:, 1:].contiguous()
    loss_fct = CrossEntropyLoss(reduction=reduction, label_smoothing=0.1)
    lm_loss = loss_fct(
        shifted_prediction_scores.view(-1, self.config.vocab_size),
        labels.view(-1),
    )

每个位置的label是它的下一个词，将预测和label计算交叉熵，得到最终损失。